In [1]:
import numpy as np
import os
import pandas as pd
from scipy.optimize import minimize, Bounds

In [2]:
n = 62
pmax = np.array([250.0, 400.0, 37.3, 300.0, 33.6])
types = np.array(['hydro', 'nuclear', 'solar', 'thermal', 'wind'])
capacity_factor = np.array([30, 95, 15, np.nan, 25])
average_load = 2800
target_energy_mix = np.array([18.75, 40.71428571, 3.99857143, 30.53714286, 6.])
target_pmax = 15000

In [3]:
def get_pmaxs(x):
  idx = np.round(x)
  pmaxs = np.zeros(types.shape)
  for i in range(types.shape[0]):
    mask = idx == i
    pmaxs[i] = pmax[i] * mask.sum()
  return pmaxs

def get_apriori_energy_mix(pmaxs, capacity_factor, average_load):
  apriori_energy_mix = capacity_factor * pmaxs / average_load
  total = np.nansum(apriori_energy_mix)
  if total > 100:
    #print("WARNING: apriori energy mix w/o thermal exceeds average load!")
    apriori_energy_mix[0] -= (total - 100)
    apriori_energy_mix[3] = 0
  else:
    apriori_energy_mix[3] = 100 - total
  return apriori_energy_mix

def split_slack_vars(x):
  n = types.shape[0] + 1
  return x[:n], x[n:]

In [4]:
def objective(x):
  slacks, _ = split_slack_vars(x)
  return slacks.sum()

# x >= 0
def constraint_1(x):
  return x

# x =< 4
def constraint_2(x):
  _, x = split_slack_vars(x)
  return -x + 4

def constraint_3(x):
  slacks, x = split_slack_vars(x)
  slacks = slacks[:-1]
  pmaxs = get_pmaxs(x)
  apriori_energy_mix = get_apriori_energy_mix(pmaxs, capacity_factor, average_load)
  return apriori_energy_mix - target_energy_mix + slacks

def constraint_4(x):
  slacks, x = split_slack_vars(x)
  slacks = slacks[-1]
  pmaxs = get_pmaxs(x)
  return -pmaxs.sum() + target_pmax - slacks

  
constraints = [
  {"type": "ineq", "fun": constraint_1},
  {"type": "ineq", "fun": constraint_2},
  {"type": "ineq", "fun": constraint_3, "tol": 1e-4},
  {"type": "ineq", "fun": constraint_4, "tol": 1e2}
]

In [5]:
def type2idx(t):
  result = np.where(types == t)
  return result[0][0]

input_path = os.path.join("..", "example", "input", "generation", "case118_l2rpn_wcci")
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))

x = np.zeros(6+n)

for i, row in df.iterrows():

  x[i+6] = type2idx(row["type"])

In [6]:
bounds = Bounds(0, np.inf)
res = minimize(objective, x, constraints=constraints, method="cobyla")
res

     fun: 2.0014285728571433
   maxcv: 2.220446049250313e-16
 message: 'Optimization terminated successfully.'
    nfev: 766
  status: 1
 success: True
       x: array([ 1.67418781e-20,  1.11236106e-19,  8.01428573e-01, -1.49178761e-19,
        1.20000000e+00,  1.25142218e-19,  3.86384159e+00,  3.86159875e+00,
        1.04741556e+00,  3.85015950e+00,  2.11979026e+00,  2.21260574e+00,
        3.41234072e-01,  2.08683736e+00,  3.14292556e+00,  2.12143062e+00,
        6.96616580e-02,  3.11732108e+00,  3.87198620e+00,  7.76995921e-01,
        3.86811870e+00,  3.88902227e+00,  3.02342586e+00,  3.80926294e+00,
        3.07068113e+00,  4.53222029e-20,  3.05244783e+00,  3.84400558e+00,
        2.11613722e+00,  2.10824100e+00,  3.73175742e+00,  2.09912593e+00,
        2.97830818e+00,  2.09959682e+00,  1.81276688e+00,  3.55850890e+00,
        2.04551366e+00,  2.07921733e+00,  2.94819937e+00,  3.97936600e+00,
        1.73453785e+00,  7.90090495e-01,  3.03142589e+00,  2.94501395e+00,
        5.212

In [7]:
x = res.x

In [8]:
constraint_1(x)

array([ 1.67418781e-20,  1.11236106e-19,  8.01428573e-01, -1.49178761e-19,
        1.20000000e+00,  1.25142218e-19,  3.86384159e+00,  3.86159875e+00,
        1.04741556e+00,  3.85015950e+00,  2.11979026e+00,  2.21260574e+00,
        3.41234072e-01,  2.08683736e+00,  3.14292556e+00,  2.12143062e+00,
        6.96616580e-02,  3.11732108e+00,  3.87198620e+00,  7.76995921e-01,
        3.86811870e+00,  3.88902227e+00,  3.02342586e+00,  3.80926294e+00,
        3.07068113e+00,  4.53222029e-20,  3.05244783e+00,  3.84400558e+00,
        2.11613722e+00,  2.10824100e+00,  3.73175742e+00,  2.09912593e+00,
        2.97830818e+00,  2.09959682e+00,  1.81276688e+00,  3.55850890e+00,
        2.04551366e+00,  2.07921733e+00,  2.94819937e+00,  3.97936600e+00,
        1.73453785e+00,  7.90090495e-01,  3.03142589e+00,  2.94501395e+00,
        5.21272743e-02,  3.01164097e+00,  3.19310991e+00,  2.83509727e+00,
        4.30973600e-02,  2.20800003e+00,  2.14878635e+00,  5.58688591e-02,
        2.92716032e+00,  

In [9]:
constraint_2(x)

array([1.36158414e-01, 1.38401245e-01, 2.95258444e+00, 1.49840503e-01,
       1.88020974e+00, 1.78739426e+00, 3.65876593e+00, 1.91316264e+00,
       8.57074437e-01, 1.87856938e+00, 3.93033834e+00, 8.82678921e-01,
       1.28013802e-01, 3.22300408e+00, 1.31881296e-01, 1.10977729e-01,
       9.76574135e-01, 1.90737055e-01, 9.29318870e-01, 4.00000000e+00,
       9.47552172e-01, 1.55994415e-01, 1.88386278e+00, 1.89175900e+00,
       2.68242583e-01, 1.90087407e+00, 1.02169182e+00, 1.90040318e+00,
       2.18723312e+00, 4.41491097e-01, 1.95448634e+00, 1.92078267e+00,
       1.05180063e+00, 2.06340002e-02, 2.26546215e+00, 3.20990950e+00,
       9.68574108e-01, 1.05498605e+00, 3.94787273e+00, 9.88359035e-01,
       8.06890086e-01, 1.16490273e+00, 3.95690264e+00, 1.79199997e+00,
       1.85121365e+00, 3.94413114e+00, 1.07283968e+00, 7.40502662e-01,
       6.35536678e-05, 1.72844579e+00, 1.64911951e+00, 6.46471813e-01,
       1.67152987e-03, 3.72990282e+00, 1.21496730e-01, 1.06114094e+00,
      

In [10]:
constraint_3(x)

array([ 1.67418781e-20,  4.28571667e-09,  2.22044605e-16,  2.00142857e+00,
       -2.22044605e-16])

In [11]:
constraint_4(x)

4915.6

In [12]:
_, x = split_slack_vars(x)

In [13]:
pmaxs = get_pmaxs(x)
get_apriori_energy_mix(pmaxs, capacity_factor, average_load)

array([18.75      , 40.71428571,  3.19714286, 32.53857143,  4.8       ])

In [14]:
pmaxs

array([1750. , 1200. ,  596.8, 6000. ,  537.6])

In [15]:
pmaxs.sum()

10084.4